# Урок 13. Модель BERT и GPT
## Задание

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации. Проверить насколько хорошо она суммаризирует.

In [1]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt
!pip install -q --upgrade datasets
!pip install -q rouge==0.3.1
!pip install -q transformers sentencepiece

     |████████████████████████████████| 431 kB 4.8 MB/s 
     |████████████████████████████████| 115 kB 63.9 MB/s 
     |████████████████████████████████| 120 kB 65.3 MB/s 
     |████████████████████████████████| 212 kB 58.2 MB/s 
     |████████████████████████████████| 127 kB 70.5 MB/s 
     |████████████████████████████████| 4.9 MB 4.8 MB/s 
     |████████████████████████████████| 1.3 MB 50.5 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 


In [2]:
from datasets import load_dataset
import json
import random
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from transformers import AutoTokenizer, MBartForConditionalGeneration
from tqdm import tqdm_notebook

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")["test"]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [5]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [6]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [7]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

In [8]:
def calc_mbart_score(records, 
                     lower=False, 
                     nrows=1000, 
                     max_length_text=600, 
                     max_length_gen_text=168):


    references = []
    predictions = []

    for record in tqdm_notebook(records[:nrows]):

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()

        input_ids = tokenizer.prepare_seq2seq_batch(
            [text],
            src_lang="ru_XX", 
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_length_text
        )["input_ids"].cuda()

        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_length_gen_text,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )[0]

        prediction = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.append(prediction)

    calc_scores(references, predictions)

In [9]:
calc_mbart_score(test_records, nrows=500)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3668: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Count: 500
Ref: В России могут ужесточить наказание за оскорбления, которые порочат честь или достоинство граждан. Соответствующее поручение Генпрокуратуре, Роскомнадзору и Верховному суду дал президент РФ Владимир Путин. Ведомствам необходимо проанализировать уголовные дела, возбужденные по факту оскорблений в интернете, и предложить варианты, как можно усилить ответственность за данное правонарушение. Предоставить отчет о проделанной работе чиновники должны 1 июля 2020 года.
Hyp: Президент России Владимир Путин поручил рассмотреть вопрос усиления ответственности за оскорбления — такое решение он принял по итогам заседания Совета по развитию гражданского общества и правам человека (СПЧ). Глава государства поручил Генпрокуратуре и Роскомнадзору до 1 июля 2020 года проанализировать практику по уголовным делам, возбужденным по факту распространения в интернете информации, «порочащей честь или достоинство гражданина». В результате ведомства должны предоставить варианты совершенствования «

Результат хороший, выше метрики были только у Oracle summary. Сам сгенерированный текст также вполне логичен и по смыслу идентичен target-summary